# 02 嵌入与语义搜索

本 Notebook 演示如何将文本转换为向量（嵌入），并利用 LanceDB 实现语义搜索。

**前置要求**：需要配置 SiliconFlow API Key（OpenAI 兼容接口）。

## 1. 什么是嵌入（Embedding）？

嵌入是将文本映射为高维向量的过程。语义相似的文本在向量空间中距离更近：

- "质量很好" 和 "品质不错" → 向量距离**近**
- "质量很好" 和 "发货太慢" → 向量距离**远**

嵌入模型（如 Qwen3-Embedding）经过大规模文本训练，能捕捉语义信息。本教程使用 SiliconFlow 提供的 `Qwen/Qwen3-Embedding-4B` 模型（1024 维）。

## 2. 配置 API

SiliconFlow 提供 OpenAI 兼容的 API 接口。请确保已设置环境变量：

```bash
export OPENAI_API_KEY='your-siliconflow-key'
export OPENAI_BASE_URL='https://api.siliconflow.cn/v1'
```

In [ ]:
import os

from openai import OpenAI

# 从环境变量读取配置
api_key = os.environ.get("OPENAI_API_KEY")
base_url = os.environ.get("OPENAI_BASE_URL", "https://api.siliconflow.cn/v1")

assert api_key, "请设置环境变量 OPENAI_API_KEY"

client = OpenAI(api_key=api_key, base_url=base_url)

EMBEDDING_MODEL = "Qwen/Qwen3-Embedding-4B"
EMBEDDING_DIM = 1024

print(f"API base_url: {base_url}")
print(f"Embedding model: {EMBEDDING_MODEL} ({EMBEDDING_DIM}d)")

## 3. 生成单条文本嵌入

先用一条文本演示嵌入生成的原理。

In [ ]:
def get_embedding(text: str) -> list[float]:
    """调用 API 生成单条文本的嵌入向量"""
    response = client.embeddings.create(input=[text], model=EMBEDDING_MODEL)
    return response.data[0].embedding


# 测试
vec = get_embedding("这个产品质量很好")
print(f"向量维度: {len(vec)}")
print(f"前 5 个分量: {vec[:5]}")

### 验证语义相似性

计算几组文本之间的余弦相似度，验证嵌入是否捕捉了语义信息。

In [ ]:
import numpy as np


def cosine_similarity(a: list[float], b: list[float]) -> float:
    """计算两个向量的余弦相似度"""
    a, b = np.array(a), np.array(b)
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))


pairs = [
    ("质量很好", "品质不错"),
    ("质量很好", "发货太慢"),
    ("手机拍照清晰", "相机画质好"),
    ("手机拍照清晰", "酒店环境差"),
]

for text_a, text_b in pairs:
    vec_a = get_embedding(text_a)
    vec_b = get_embedding(text_b)
    sim = cosine_similarity(vec_a, vec_b)
    print(f"  '{text_a}' vs '{text_b}' → 相似度: {sim:.4f}")

## 4. 批量生成嵌入并写入 LanceDB

加载评论数据集，批量生成嵌入向量，存入 LanceDB。

In [ ]:
import pandas as pd

reviews = pd.read_csv("../data/reviews.csv")
print(f"数据集大小: {len(reviews):,} 条")
reviews.head()

### 批量调用 API 生成嵌入

OpenAI 兼容接口支持批量输入，每次最多传入多条文本。我们按 batch 分批处理以避免超时。

In [ ]:
def get_embeddings_batch(texts: list[str], batch_size: int = 64) -> list[list[float]]:
    """分批生成嵌入向量"""
    all_embeddings: list[list[float]] = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        response = client.embeddings.create(input=batch, model=EMBEDDING_MODEL)
        batch_embeddings = [item.embedding for item in response.data]
        all_embeddings.extend(batch_embeddings)
        print(f"  已处理 {min(i + batch_size, len(texts))}/{len(texts)}")
    return all_embeddings


print("正在生成嵌入向量...")
embeddings = get_embeddings_batch(reviews["review"].tolist())
print(f"完成，共 {len(embeddings)} 个向量，维度 {len(embeddings[0])}")

### 写入 LanceDB

将评论数据和嵌入向量一起写入 LanceDB 表。

In [ ]:
import lancedb

# 添加向量列
reviews["vector"] = embeddings

# 连接数据库并创建表
db = lancedb.connect("../lancedb_data")
table = db.create_table("reviews", reviews.to_dict("list"), mode="overwrite")

print(f"表名: {table.name}")
print(f"行数: {table.count_rows()}")
print(f"Schema: {table.schema}")

## 5. 语义搜索

输入一段查询文本，找到语义最相似的评论。

In [ ]:
def semantic_search(query: str, top_k: int = 5) -> pd.DataFrame:
    """语义搜索：输入查询文本，返回最相似的评论"""
    query_vec = get_embedding(query)
    results = table.search(query_vec).limit(top_k).to_pandas()
    return results


# 搜索正面评价
results = semantic_search("质量好，非常满意")
print("查询: '质量好，非常满意'\n")
for _, row in results.iterrows():
    print(f"  [{row['cat']}] (label={row['label']}) {row['review'][:60]}")
    print(f"    距离: {row['_distance']:.4f}")

In [ ]:
# 搜索负面评价
results = semantic_search("太差了，不推荐购买")
print("查询: '太差了，不推荐购买'\n")
for _, row in results.iterrows():
    print(f"  [{row['cat']}] (label={row['label']}) {row['review'][:60]}")
    print(f"    距离: {row['_distance']:.4f}")

In [ ]:
# 搜索特定主题
results = semantic_search("手机拍照效果")
print("查询: '手机拍照效果'\n")
for _, row in results.iterrows():
    print(f"  [{row['cat']}] (label={row['label']}) {row['review'][:60]}")
    print(f"    距离: {row['_distance']:.4f}")

## 6. 混合搜索：向量 + 过滤

在语义搜索的基础上，添加标量过滤条件（如按类别、按情感标签过滤）。

In [ ]:
def hybrid_search(
    query: str,
    cat: str | None = None,
    label: int | None = None,
    top_k: int = 5,
) -> pd.DataFrame:
    """混合搜索：语义搜索 + 标量过滤"""
    query_vec = get_embedding(query)
    search = table.search(query_vec).limit(top_k)

    filters: list[str] = []
    if cat is not None:
        filters.append(f"cat = '{cat}'")
    if label is not None:
        filters.append(f"label = {label}")
    if filters:
        search = search.where(" AND ".join(filters))

    return search.to_pandas()

In [ ]:
# 只搜索 "手机" 类别的正面评论
results = hybrid_search("屏幕显示效果", cat="手机", label=1)
print("查询: '屏幕显示效果' (类别=手机, 正面)\n")
for _, row in results.iterrows():
    print(f"  [{row['cat']}] (label={row['label']}) {row['review'][:60]}")

In [ ]:
# 搜索 "酒店" 类别的负面评论
results = hybrid_search("服务态度", cat="酒店", label=0)
print("查询: '服务态度' (类别=酒店, 负面)\n")
for _, row in results.iterrows():
    print(f"  [{row['cat']}] (label={row['label']}) {row['review'][:60]}")

## 7. 创建索引优化搜索性能

当数据量较大时，可以创建 IVF-PQ 索引来加速搜索。索引将向量空间划分为多个分区，搜索时只扫描最相关的分区。

> 注意：当前数据集只有 3000 条，索引的加速效果不明显。数据量达到万级以上时效果更显著。

In [ ]:
import time

# 无索引搜索
query_vec = get_embedding("质量好")

start = time.time()
for _ in range(10):
    table.search(query_vec).limit(5).to_list()
no_index_time = (time.time() - start) / 10
print(f"无索引搜索: {no_index_time*1000:.2f} ms/次")

In [ ]:
# 创建 IVF-PQ 索引
table.create_index(
    metric="L2",
    num_partitions=16,
    num_sub_vectors=32,
)
print("索引创建完成")

In [ ]:
# 有索引搜索
start = time.time()
for _ in range(10):
    table.search(query_vec).limit(5).to_list()
index_time = (time.time() - start) / 10
print(f"有索引搜索: {index_time*1000:.2f} ms/次")
print(f"加速比: {no_index_time/index_time:.2f}x")

## 小结

本 Notebook 介绍了：
- 嵌入的概念：文本 → 高维向量，语义相似的文本向量距离近
- 使用 OpenAI 兼容 API（SiliconFlow）生成嵌入
- 批量生成嵌入并写入 LanceDB
- 语义搜索：根据含义而非关键词查找相似文本
- 混合搜索：向量搜索 + 标量过滤
- IVF-PQ 索引：加速大规模向量搜索

下一个 Notebook 将展示如何用 Daft 与 LanceDB 集成，构建完整的数据处理 + 嵌入 + 搜索 pipeline。